<a href="https://colab.research.google.com/github/indrajitkurmi/aos-scripts/blob/main/MSE_with_Motion_DRIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# utility function(s)
def imshow(image, *args, **kwargs):
    """A replacement for cv2.imshow() for use in Jupyter notebooks using matplotlib.

        Args:
          image : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
            (N, M, 3) is an NxM BGR color image. 
    """
    if len(image.shape) == 3:
      # Height, width, channels
      # Assume BGR, do a conversion  
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Draw the image
    plt.imshow(image, *args, **kwargs)
    # We'll also disable drawing the axes and tick marks in the plot, since it's actually an image
    plt.axis('off')
    # Make sure it outputs
    # plt.show()

A single binary view (area: $w \times w$) is composed of zero background and of a target of size $a \times a$. The mean and variance of such a 2D view are 
$$
\mu = \frac{a^2}{w^2} \\
\sigma = \mu(\mu-1)^2 + (1-\mu)(\mu-0)^2 =  \mu - \mu^2
$$
, where the mean ($\mu$) also describes the size relation of the target with respect to the view. The term $(\mu-1)^2$ is the variance computation in the target and $(\mu-0)^2 = \mu^2$ the variance for the background.

The code below shows such an example:

In [ ]:
w = 300
a =  50
sx,sy = int(w/3), int(w/3)

single = np.zeros((300,300))
single[sx:sx+a,sy:sy+a] = 1.0

imshow(single)
plt.title( 'mean: {}, var: {}'.format(np.mean(single), np.var(single)))
plt.show()

If multiple such single views are averaged (as typically done in AOS) it has no effect on the variance and mean as long as the target is perfectly registered. 

If the target, however, is not registered (e.g., a moving target or by defocus) the statistics change. 
Let's first look at the extreme case, where the averaged targets are not overlapping anymore. This is showcased below by introducing a shift $d$ for $N$ images.
The mean ($\mu$) is not altered, but the variance changes:
$$
\sigma = N\mu(\mu-\frac{1}{N})^2 + (1-N\mu)(\mu-0)^2 =  \frac{\mu}{N} - \mu^2 \text{.}
$$
The change of $\sigma$ is inverse proportional to $N$. Note that $N\mu$ describes the area covered by the non-overlapping instances of the target. 

In [ ]:
w = 300
a =  20
r = a*a / w**2 # ratio/mean
print(r)
d = 25
N = 10
sx,sy = int(a), int(a)
sum = np.zeros((w,w))

for i in range(N):
  single = np.zeros((w,w))
  x,y = sx + i*d, sy
  single[x:x+a,y:y+a] = 1.0
  sum += single

sum = sum/N

imshow(sum, vmin=0.0, vmax=1.0)
plt.title( 'mean: {}, var: {}'.format(np.mean(sum), np.var(sum)))
plt.show()

# variance computation
v_overlap = r - r**2
v_nonoverlap = r/N - r**2
print('var (non-overlap): {}'.format( v_nonoverlap ) )


If the shift $d$ is less then the target size $a$ $(d < a)$ the targets will overlap in the integral image. For simplicity we will just look at the problem in 1D now. 
The area (normalized by the area size) that is covered by the non-overlapping targets can be expressed by 
$$
  g = \frac{d(N-1)+a}{w}
$$
and the number of overlaps is expressed by 
$$
  M = \frac{a}{d}
$$
, where it has to be ensured that $M$ does not exceed $N$. 
Furthermore, there will be different regions with a varying amount of overlap. For example a target with $a=5$ a shift of $d=2$ and $N=7$ results in 4 regions without overlap in 8 regions where two target instances overlap and 5 regions with an overlap of three targets. Note that this is illustrated in the example below. Furthermore, in this simulation a region is a pixel or array cell. 

To compute the variance the different overlaps have to be considered. We introduce this as a count $c_i$, where $i$ is the number of overlapping targets. In the example this results in $c_1=4, c_2=8$, and $c_3=5$.
The equation to compute the variance thus expands to
$$
  \sigma = (1-g)\mu^2 + \frac{1}{w} \sum_i c_i (\mu - \frac{i}{N})^2 \\
   =  \mu^2 - \frac{2\mu}{Nw} \sum_i c_i i + \frac{1}{N^2w} \sum_i c_i i^2 \text{.}
$$
By subsituting $\mu = a/w$ (in 1D) it further simplifies to
$$
  \sigma =  \frac{a^2}{w^2} - \frac{2a}{Nw^2} \sum_i{ c_i i }+ \frac{1}{N^2w} \sum_i c_i i^2 \text{.}
$$

Note that it is propably impractical to always compute $c_i$ so it might be possible to simplify or approximate these terms. A first attempt would be to approximate the terms by
$$
 \sum_i{ c_i i }   \approx M (d (N-1-M)+a) \\
 \sum_i{ c_i i^2 } \approx M^2 (d (N-1-M)+a) \text{.}
$$
This, however does not allways lead to close results (see below). 



In [ ]:
w = 30
a =  5
r = a / w # ratio/mean
print(r)
d = 2
N = 7
sx,sy = int(a), int(a)
sum = np.zeros((1,w))

for i in range(N):
  single = np.zeros_like(sum)
  x,y = sx + i*d, sy
  single[:,x:x+a] = 1.0
  sum += single

count,bins=np.histogram(sum, bins=np.arange(np.max(sum)+2))
print(count) 
print(np.asarray(bins[:-1],dtype=np.int16))
sum = sum/N

imshow(sum, vmin=0.0, vmax=1.0)
plt.title( 'mean: {}, var: {}'.format(np.mean(sum), np.var(sum)))
plt.show()

# variance computation
if d<=0:
  M = N
else:
  M = max(min(a/d,N),1)

v_overlap = r - r**2        # assuming everything is overlapping
v_nonoverlap = r/N - r**2   # assuming nothing is overlapping in the integral

term1 = np.sum(bins[:-1] * count)
term2 = np.sum(bins[:-1]**2 * count)
v = a**2/w**2 - 2*a/(N*w**2)*term1 + 1/(N**2*w)*term2
print('var (new): {}'.format( v ) )

# approximate term1 and term2
term1_ = M * (d*(N-1-M)+a)
term2_ = term1_ * M
v_ = a**2/w**2 - 2*a/(N*w**2)*term1_ + 1/(N**2*w)*term2_
print('var (approx): {}'.format( v_ ) )



In this section, we discuss the statistical model of the
$MSE$ between an integral image $X$ and an hypothetical
occlusion-free reference $S$ . 
$$
MSE = E[(X- S)^2] = E[X^2] -2E[XS] +E[S^2]
$$
Currently assuming ground also as an occluder with same mean and variance the occluders the integral image X can be defined as :-
$$
X = \frac{1}{N}\bigg[\sum_{i=1}^{A} Z_iO_i + (1- Z_i)S + \sum_{i = A+1}^{N} O_i \bigg]
$$
Thus
$$
MSE = \bigg[\bigg(1 - \frac{A(1-D)}{N}\bigg)^2 + \frac{AD(1-D)}{N^2}\bigg] \bigg(\sigma_s^2 + (\mu_o - \mu_s)^2\bigg) +  \frac{AD + N - A}{N^2} \sigma_o^2
$$

Below is theorithical MSE calulation provided ( signal mean, siganl variance, occluder mean, occluder variance, occluder density, no of integrated images, numof overlapping images)

--- Current Equation works when number of overlapping images images are integers ( i.e image size is multiple of shift)

In [ ]:
def theoritcal_MSE(signalmean,signalvar,occlmean,occlvar,occldens,noofintegratedimage,numofover):
  MSE = ((1-(numofover*(1-occldens)/noofintegratedimage))**2 + (numofover*occldens*(1-occldens)/(noofintegratedimage**2)))*(signalvar+(occlmean-signalmean)**2) + (((numofover*occldens+noofintegratedimage-numofover)/(noofintegratedimage**2))*occlvar)
  return MSE

In [ ]:
def theoritcal_MSE_Parallel_Sequential(signalmean,signalvar,occlmean,occlvar,occldens,noofintegratedimage,numofover, numofparallel):
  numofover = numofover * numofparallel
  MSE = ((1-(numofover*(1-occldens)/noofintegratedimage))**2 + (numofover*occldens*(1-occldens)/(noofintegratedimage**2)))*(signalvar+(occlmean-signalmean)**2) + (((numofover*occldens+noofintegratedimage-numofover)/(noofintegratedimage**2))*occlvar)
  return MSE

Below we measure mse when no occlusion is present in 1D Case

In [ ]:
w = 100
a =  15
r = a / w # ratio/mean
print(r)
d = 5
N = 10
signalmean = 0.5
sx,sy = int(a), int(a)
sum = np.zeros((1,w))
singleimagearray =[]
for i in range(N):
  single = np.zeros_like(sum)
  x,y = sx + i*d, sy
  single[:,x:x+a] = signalmean
  imshow(single, vmin=0.0, vmax=1.0)
  plt.show()
  singleimagearray.append(single)
  sum += single
sum = sum/N
imshow(sum, vmin=0.0, vmax=1.0)
plt.title( 'mean: {}, var: {}'.format(np.mean(sum), np.var(sum)))
plt.show()
#####Calculate Mean square error#############
noofpix = d
x,y = sx + 0*d, sy
endx,endy = sx + (N-2)*d, sy
imshow(sum[:,x+a-noofpix:endx+a-noofpix], vmin=0.0, vmax=1.0)
plt.title( 'Integrated Signal mean: {}, var: {}'.format(np.mean(sum[:,x+a-noofpix:endx+a-noofpix]), np.var(sum[:,x+a-noofpix:endx+a-noofpix])))
plt.show() 
nopa = sum[:,x+a-noofpix:endx+a-noofpix]
nop = len(nopa)
sourcesingle = np.zeros((1,w))
sourcesingle[:,x+a-noofpix:endx+a-noofpix] = signalmean
imshow(sourcesingle[:,x+a-noofpix:endx+a-noofpix], vmin=0.0, vmax=1.0)
plt.title( 'Source Signal mean: {}, var: {}'.format(np.mean(sourcesingle[:,x+a-noofpix:endx+a-noofpix]), np.var(sourcesingle[:,x+a-noofpix:endx+a-noofpix])))
plt.show() 
calcmse = np.mean((sum[:,x+a-noofpix:endx+a-noofpix] - sourcesingle[:,x+a-noofpix:endx+a-noofpix])**2)
print("calculated mse",calcmse)
theoriticalmse = theoritcal_MSE(0.5,0,0,0,0,N,int(a/d))
print("theoritical mse",theoriticalmse)

Below we show equation validation for binary occluders in 1D shift when computed with a 2D image

In [ ]:
max_no_integrated_images = 50
occl_density = 0.15
occl_size = 5
occl_disparity = 5
motion_shift = 20
imgSize = (2048,2048)
integral_image = np.zeros(imgSize)
mixType = 'replace'
signalType = 'binarymotion'
signalMean = 0.0
signalSigma = 0
signalsize = (400,400)
noiseType = 'binary'
noiseSigma = 0
noiseMean = 1
N = 10
if signalType == 'binarymotion':
  signal = np.ones(imgSize) # create signal filled with ones and create a signal region in the image
  signal[int(np.floor(imgSize[0]/2-signalsize[0]/2)):int(np.ceil(imgSize[0]/2+signalsize[0]/2)),int(np.floor((signalsize[1]+1)-signalsize[1]/2)):int(np.ceil((signalsize[1]+1)+signalsize[1]/2))] = signalMean
imshow(signal, vmin=0.0, vmax=1.0)
plt.show()
## To Check Moving  Signal
#for i in range(1,max_no_integrated_images):
#  rotsignal = np.roll(signal, i*motion_shift, axis=1)
#  imshow(rotsignal, vmin=0.0, vmax=1.0)
#  plt.show()
mse = []
theo_mse = []
singleimage_stack = []
noiseImgSize = (int(np.ceil( imgSize[0]/occl_size + (max_no_integrated_images*occl_disparity))),int(np.ceil( imgSize[1]/occl_size + max_no_integrated_images*occl_disparity)))
print(noiseImgSize)
#create uniformly distributed random image filled with ones
uniform_rand_img = (np.random.uniform(low = 0.0, high = 1.0,size = noiseImgSize) <= occl_density).astype(int) * noiseMean
#resize the image to create occluders of size occl_size
shiftImg = cv2.resize(src = uniform_rand_img, dsize = (noiseImgSize[0]*occl_size,noiseImgSize[1]*occl_size), interpolation = cv2.INTER_NEAREST)
#To check if occluders are binary with noise mean
nonzerosimg = shiftImg[np.nonzero(shiftImg)]
print('min: {}, max: {}'.format(min(nonzerosimg), max(nonzerosimg)))
#imshow(shiftImg, vmin=0.0, vmax=1.0)
#plt.title( 'mean: {}, var: {}'.format(np.mean(shiftImg), np.var(shiftImg)))
#plt.show()
summedimage = np.zeros(imgSize)
for i in range(1,max_no_integrated_images):
  pixShift = i * occl_disparity
  tmp = np.zeros(imgSize)
  #Take a shifted portion of the noise image
  tmp = shiftImg[0:imgSize[0],pixShift+0:pixShift+imgSize[1]]
  #shift the signal to show the motion
  signal = np.roll(signal, motion_shift, axis=1)
  #replace where noise is zero
  combimg = tmp.copy()
  combimg[tmp == 0] = signal[tmp == 0]
  
  #combimg = tmp + signal
  #combimg[combimg>=noiseMean] = noiseMean
  #imshow(combimg, vmin=0.0, vmax=1.0)
  #plt.show()
  #imshow(signal, vmin=0.0, vmax=1.0)
  #plt.show()
  #add to the sum image
  summedimage = summedimage + combimg
  # divide by i to get the mean integral image
  integral_image = summedimage / i
  #calculate start and end pos of area for which mse is calulated ---
  # For N < A we take image regions where N signal images are integrated
  # For N > A we take image regions where A signal images are integrated
  startpos = np.floor((signalsize[1]+1)-signalsize[1]/2) + (min(i, np.ceil(signalsize[1]/motion_shift))) * motion_shift
  endpos =  np.floor((signalsize[1]+1)+signalsize[1]/2) + (max(i-np.ceil(signalsize[1]/motion_shift),1)) * motion_shift
  # Copy the selected region from the integral image
  projimg = integral_image[int(np.floor(imgSize[0]/2-signalsize[0]/2)):int(np.ceil(imgSize[0]/2+signalsize[0]/2)),int(startpos):int(endpos)]
  # create a binary signal image of same region
  sigimg = np.ones(projimg.shape)*signalMean
  # calculate mse
  squared_subtractimg = np.square(np.subtract(projimg,sigimg))
  avg = np.mean(squared_subtractimg)
  # For N < A we take A = N
  # For N >= A we take A
  noofimageoverlap = min(i, np.ceil(signalsize[1]/motion_shift));
  # Calculate  theoritical mse
  theoriticalmse = theoritcal_MSE(signalMean,signalSigma,noiseMean,noiseSigma,occl_density,i,noofimageoverlap)
  print('measured mse: {}, theoritical mse: {}'.format(avg, theoriticalmse))
  mse.append(avg)
  theo_mse.append(theoriticalmse)
plt.plot(range(1,max_no_integrated_images), mse, 'g--', linewidth=2, markersize=12 , label = 'measured mse')
plt.plot(range(1,max_no_integrated_images), theo_mse, 'r', linewidth=2, markersize=12 , label = 'theoritical mse')
plt.legend()
plt.show()  

In [ ]:
w = 100
a =  20
r = a / w # ratio/mean
print(r)
d = 2
o_size = 2
o_shift = 2
o_dens = 0.7
N = 20

sx,sy = int(a), int(a)
sum = np.zeros((1,w))
singleimagearray =[]
occlimage = np.random.choice([0, 1], size=(1,50), p=[o_dens, 1-o_dens]) #np.random.binomial(n=1, p=1-o_dens, size=(1,50))
print(np.mean(occlimage), np.count_nonzero(occlimage), np.count_nonzero(occlimage)/50)
imshow(occlimage, vmin=0.0, vmax=1.0)
plt.show()
occlimage = occlimage.repeat(o_size, axis=1)
meas_dens = 1 - np.count_nonzero(occlimage)/100
print(np.mean(occlimage), np.count_nonzero(occlimage),np.count_nonzero(occlimage)/100)
imshow(occlimage, vmin=0.0, vmax=1.0)
plt.show()
shiftedocclimage = np.roll(occlimage, 2)
imshow(shiftedocclimage, vmin=0.0, vmax=1.0)
plt.show()
for i in range(N):
  single = np.zeros_like(sum)
  x,y = sx + i*d, sy
  single[:,x:x+a] = 0.5
  shiftedocclimage = np.roll(occlimage, i*o_shift)
  imshow(shiftedocclimage, vmin=0.0, vmax=1.0)
  plt.show()
  single = single * shiftedocclimage
  imshow(single, vmin=0.0, vmax=1.0)
  plt.show()
  singleimagearray.append(single)
  sum += single
sum = sum/N
imshow(sum, vmin=0.0, vmax=1.0)
plt.title( 'mean: {}, var: {}'.format(np.mean(sum), np.var(sum)))
plt.show()
#####Calculate Mean square error#############
noofpix = d
x,y = sx + 0*d, sy
endx,endy = sx + (N-4)*d, sy
imshow(sum[:,x+a-noofpix:endx+a-noofpix], vmin=0.0, vmax=1.0)
plt.title( 'Integrated Signal mean: {}, var: {}'.format(np.mean(sum[:,x+a-noofpix:endx+a-noofpix]), np.var(sum[:,x+a-noofpix:endx+a-noofpix])))
plt.show() 
nopa = sum[:,x+a-noofpix:endx+a-noofpix]
nop = len(nopa)
sourcesingle = np.zeros((1,w))
sourcesingle[:,x+a-noofpix:endx+a-noofpix] = signalmean
imshow(sourcesingle[:,x+a-noofpix:endx+a-noofpix], vmin=0.0, vmax=1.0)
plt.title( 'Source Signal mean: {}, var: {}'.format(np.mean(sourcesingle[:,x+a-noofpix:endx+a-noofpix]), np.var(sourcesingle[:,x+a-noofpix:endx+a-noofpix])))
plt.show() 
calcmse = np.mean((sum[:,x+a-noofpix:endx+a-noofpix] - sourcesingle[:,x+a-noofpix:endx+a-noofpix])**2)
print("calculated mse",calcmse)
theoriticalmse = theoritcal_MSE(0.5,0,0,0,meas_dens,N,int(a/d))
print("theoritical mse",theoriticalmse)

Below we measure when occlusion are randomly present with Density D

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
nums = (np.random.uniform(low = 0.0, high = 1.0,size = (40,40)) <= 0.5).astype(int)
print(nums)
print(np.mean(nums), np.count_nonzero(nums)/1000)

In [ ]:
max_no_integrated_images = 201
occl_density = 0.5
occl_size = 5
occl_disparity = 5
motion_shift = 50
num_parallel_camera = 5
imgSize = (2048,2048)
integral_image = np.zeros(imgSize)
mixType = 'replace'
signalType = 'binarymotion'
signalMean = 0.0
signalSigma = 0
signalsize = (400,400)
noiseType = 'binary'
noiseSigma = 0
noiseMean = 1
N = 10
if signalType == 'binarymotion':
  signal = np.ones(imgSize) # create signal filled with ones and create a signal region in the image
  signal[int(np.floor(imgSize[0]/2-signalsize[0]/2)):int(np.ceil(imgSize[0]/2+signalsize[0]/2)),int(np.floor((signalsize[1]+1)-signalsize[1]/2)):int(np.ceil((signalsize[1]+1)+signalsize[1]/2))] = signalMean
imshow(signal, vmin=0.0, vmax=1.0)
plt.show()
## To Check Moving  Signal
#for i in range(1,max_no_integrated_images):
#  rotsignal = np.roll(signal, i*motion_shift, axis=1)
#  imshow(rotsignal, vmin=0.0, vmax=1.0)
#  plt.show()
mse = []
theo_mse = []
singleimage_stack = []
noiseImgSize = (int(np.ceil( imgSize[0]/occl_size + (max_no_integrated_images*occl_disparity))),int(np.ceil( imgSize[1]/occl_size + max_no_integrated_images*occl_disparity)))
print(noiseImgSize)
#create uniformly distributed random image filled with ones
uniform_rand_img = (np.random.uniform(low = 0.0, high = 1.0,size = noiseImgSize) <= occl_density).astype(int) * noiseMean
#resize the image to create occluders of size occl_size
shiftImg = cv2.resize(src = uniform_rand_img, dsize = (noiseImgSize[0]*occl_size,noiseImgSize[1]*occl_size), interpolation = cv2.INTER_NEAREST)
#To check if occluders are binary with noise mean
nonzerosimg = shiftImg[np.nonzero(shiftImg)]
print('min: {}, max: {}'.format(min(nonzerosimg), max(nonzerosimg)))
#imshow(shiftImg, vmin=0.0, vmax=1.0)
#plt.title( 'mean: {}, var: {}'.format(np.mean(shiftImg), np.var(shiftImg)))
#plt.show()
summedimage = np.zeros(imgSize)
numberofmotionshift = 1
for i in range(1,max_no_integrated_images):
  pixShift = i * occl_disparity
  tmp = np.zeros(imgSize)
  #Take a shifted portion of the noise image
  tmp = shiftImg[0:imgSize[0],pixShift+0:pixShift+imgSize[1]]
  print('i',i)
  if (i-1) % num_parallel_camera == 0:
    print('shifted signal')
    #shift the signal to show the motion
    signal = np.roll(signal, motion_shift, axis=1)
    numberofmotionshift = numberofmotionshift + 1
    #replace where noise is zero
  combimg = tmp.copy()
  combimg[tmp == 0] = signal[tmp == 0]
  
  #combimg = tmp + signal
  #combimg[combimg>=noiseMean] = noiseMean
  #imshow(combimg, vmin=0.0, vmax=1.0)
  #plt.show()
  #imshow(signal, vmin=0.0, vmax=1.0)
  #plt.show()
  #add to the sum image
  summedimage = summedimage + combimg
  # divide by i to get the mean integral image
  integral_image = summedimage / i
  if i % num_parallel_camera == 0:
    #calculate start and end pos of area for which mse is calulated ---
    # For N < A we take image regions where N signal images are integrated
    # For N > A we take image regions where A signal images are integrated
    startpos = np.floor((signalsize[1]+1)-signalsize[1]/2) + (min(numberofmotionshift, np.ceil(signalsize[1]/motion_shift))) * motion_shift
    endpos =  np.floor((signalsize[1]+1)+signalsize[1]/2) + (max(numberofmotionshift-np.ceil(signalsize[1]/motion_shift),1)) * motion_shift
    # Copy the selected region from the integral image
    projimg = integral_image[int(np.floor(imgSize[0]/2-signalsize[0]/2)):int(np.ceil(imgSize[0]/2+signalsize[0]/2)),int(startpos):int(endpos)]
    # create a binary signal image of same region
    sigimg = np.ones(projimg.shape)*signalMean
    print('Startpos',startpos,'endpse',endpos)
    imshow(integral_image, vmin=0.0, vmax=1.0)
    plt.show()
    imshow(projimg, vmin=0.0, vmax=1.0)
    plt.show()
    imshow(sigimg, vmin=0.0, vmax=1.0)
    plt.show()
    # calculate mse
    squared_subtractimg = np.square(np.subtract(projimg,sigimg))
    avg = np.mean(squared_subtractimg)
    # For N < A we take A = N
    # For N >= A we take A
    noofimageoverlap = min((numberofmotionshift-1), np.ceil(signalsize[1]/motion_shift))
    # Calculate  theoritical mse
    theoriticalmse = theoritcal_MSE_Parallel_Sequential(signalMean,signalSigma,noiseMean,noiseSigma,occl_density,i,noofimageoverlap,num_parallel_camera)
    print('measured mse: {}, theoritical mse: {}'.format(avg, theoriticalmse))
    mse.append(avg)
    theo_mse.append(theoriticalmse)
plt.plot(range(1,numberofmotionshift), mse, 'g--', linewidth=2, markersize=12 , label = 'measured mse')
plt.plot(range(1,numberofmotionshift), theo_mse, 'r', linewidth=2, markersize=12 , label = 'theoritical mse')
plt.legend()
plt.show()  